# 单机版PyTorch任务

本例介绍了使用TI SDK训练PyTorch模型

#### 引入依赖

In [1]:
from __future__ import absolute_import, print_function

import sys
from ti import session
from ti.pytorch import PyTorch

#### 准备训练数据和脚本

我们已经在对象存储COS中准备了一份数据集，您也可以替换为自己的数据。

**测试数据集**(点击[此链接](https://ti-ap-guangzhou-1300268737.cos.ap-guangzhou.myqcloud.com/training_data/pytorch/simple/test)下载)

**训练数据集**(点击[此链接](https://ti-ap-guangzhou-1300268737.cos.ap-guangzhou.myqcloud.com/training_data/pytorch/simple/train)下载)

**验证数据集**(点击[此链接](https://ti-ap-guangzhou-1300268737.cos.ap-guangzhou.myqcloud.com/training_data/pytorch/simple/valid)下载)


#### 初始化PyTorch Estimator

In [ ]:
# 初始化session
ti_session = session.Session()

# 授权给TI的服务角色
role = "TIONE_QCSRole"

# COS中的训练数据
inputs = 'cos://ti-ap-guangzhou-1300268737/training_data/pytorch/simple/'

# 创建一个PyTorch Estimator
estimator = PyTorch(entry_point='train.py',
                    role=role,
                    framework_version='1.1.0',
                    train_instance_count=1,
                    train_instance_type='TI.SMALL2.1core2g',
                    source_dir='code',
                    # available hyperparameters: emsize, nhid, nlayers, lr, clip, epochs, batch_size,
                    #                            bptt, dropout, tied, seed, log_interval
                    hyperparameters={
                        'epochs': 1,
                        'tied': True
                    })

# 提交PyTorch训练任务
estimator.fit({'training': inputs})

## 查看输出模型

如果训练任务完成有模型输出，那么TI会将模型上传到cos上。
你可以通过**output_path='cos://'**指定模型存储的cos路径。如果没指定，TI会按以下格式创建存储通
**ti-[region]-[uin]-[appid]**

最终模型会放在 #{bucket}/#{job_name}/output/目录下

## 查看训练日志

TI训练过程中会把日志上传到[腾讯云日志服务](https://cloud.tencent.com/product/cls)中，使用腾讯云日志服务需要先开通。

TI会针对训练任务创建TiOne日志集和TrainingJob日志主题，通过日志检索功能可以搜索对应训练任务的日志。


目前TI会默认创建一个日志集(TI)和日志主题(TrainingJob)

TI内置了任务名称(job)关键词，可以通过以下条件过滤指定任务的日志

```
job: #{job_name}
```

更多日志检索语法请参考https://cloud.tencent.com/document/product/614/16981

## 查看提交历史和监控

您可以在TI产品控制台上的【任务列表】-【SDK任务】中查看您提交的SDK任务历史，点击【任务名称】列中的任务名可以看到任务的详细信息，在列表的右侧【监控】可以查看任务运行的资源监控图表